## SPH Modelling

In [1]:
import numpy as np
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import cv2

# Constants
N     = 500                   # Number of particles
t     = 0                     # start time of simulation
tEnd  = 8                     # end time for simulation
dt    = 0.01                  # timestep
M     = 2                     # star mass
R     = 0.75                  # star radius
h     = 0.1                   # smoothing length
k     = 0.1                   # equation of state constant
n     = 1                     # polytropic index
nu    = 100                     # damping - simulates viscosity
m     = M/N                   # single particle mass
lmbda = 2.01                  # lambda for gravity
Nt    = int(np.ceil(tEnd/dt)) # number of timesteps

# Initial Conditions (randomize in 3D)
def initial():
    np.random.seed(42)                # set random number generator seed
    pos = np.random.randn(N, 3)       # 3D positions
    vel = np.zeros(pos.shape)         # 3D velocities
    return pos, vel

#### Kernel Function

In [2]:
# Gaussian Smoothing Kernel for 3D
def kernel(x, y, z, h):
  """
  Input:
    x : matrix of x positions
    y : matrix of y positions
    h : smoothing length

  Output:
    w : evaluated smoothing function
  """
  r = np.sqrt(x**2 + y**2 + z**2)
  w = (1.0 / (h * np.sqrt(np.pi)))**3 * np.exp(-r**2 / h**2)
  return w
  
# Smoothing Kernel Gradient for 3D
def gradKernel(x, y, z, h):
  """
  Inputs:
    x : matrix of x positions
    y : matrix of y positions
    h : smoothing length

  Outputs:
    wx, wy : the evaluated gradient
  """
  r = np.sqrt(x**2 + y**2 + z**2)
  n = -2 * np.exp(-r**2 / h**2) / h**5 / (np.pi)**(3/2)
  wx = n * x
  wy = n * y
  wz = n * z
  return wx, wy, wz

  

#### Denisty Calculation

In [3]:
#rho_i = sum_j m_j W(|r_i - r_j|, h)
#From this formula, we can see that we need to find the distance between all particles to get the density at a 
# particular position. Hence, we can get the density at every position as shown below.

# Solve for the (r_i - r_j) term in the density formula
def magnitude(ri, rj):
  """
  Inputs:
    ri : M x 2 matrix of positions
    rj : N x 2 matrix of positions
  
  Output:
    dx, dy : M x N matrix of separations
  """
  M = ri.shape[0]
  N = rj.shape[0]
  r_i_x, r_i_y, r_i_z = ri[:, 0].reshape((M, 1)), ri[:, 1].reshape((M, 1)), ri[:, 2].reshape((M, 1))
  r_j_x, r_j_y, r_j_z = rj[:, 0].reshape((N, 1)), rj[:, 1].reshape((N, 1)), rj[:, 2].reshape((N, 1))
  dx = r_i_x - r_j_x.T
  dy = r_i_y - r_j_y.T
  dz = r_i_z - r_j_z.T
  return dx, dy, dz


# Get density at sample points (3D)
def density(r, pos, m, h):
  """
  Inputs:
    r   : M x 3 matrix of sampling locations
    pos : N x 3 matrix of particle positions
    m   : particle mass
    h   : smoothing length

  Output:
    rho : M x 1 vector of densities
  """
  M = r.shape[0]
  dx, dy, dz = magnitude(r, pos)
  rho = np.sum(m * kernel(dx, dy, dz, h), 1).reshape((M, 1))
  return rho

#### Pressure Calculation

In [4]:
#Find Pressure
def pressure(rho, k, n):
  """
  Inputs:
    rho : M x 1 matrix of densities
    k   : equation of state constant
    n   : polytropic index

  Output:
    P   : M x 1 matrix of pressures
  """
  P = k * rho**(1 + 1/n)
  return P

#### Acceleration Calculation

In [5]:
# Calculate acceleration on particles
def acceleration(pos, vel, m, h, k, n, lmbda, nu):
  """
  Inputs:
    pos   : N x 2 matrix of positions
    vel   : N x 2 matrix of velocities
    m     : particle mass
    h     : smoothing length
    k     : equation of state constant
    n     : polytropic index
    lmbda : external force constant
    nu    : viscosity

  Output:
    a : N x 3 matrix of accelerations
  """
  
  N = pos.shape[0]
  
  # Calculate densities
  rho = density(pos, pos, m, h)
  
  # Get pressures
  P = pressure(rho, k, n)
  
  # Get pairwise distances and gradients
  dx, dy, dz = magnitude(pos, pos)
  dWx, dWy, dWz = gradKernel(dx, dy, dz, h)
  
  # Add Pressure contribution to accelerations
  ax = - np.sum(m * (P/rho**2 + P.T/rho.T**2) * dWx, 1).reshape((N, 1))
  ay = - np.sum(m * (P/rho**2 + P.T/rho.T**2) * dWy, 1).reshape((N, 1))
  az = - np.sum(m * (P/rho**2 + P.T/rho.T**2) * dWz, 1).reshape((N, 1))
  
  # Pack acceleration components
  a = np.hstack((ax,ay,az))
  
  # Add external forces
  a += -lmbda * pos - nu * vel
  
  # Return total acceleration
  return a

#### Main Loop Simulation

In [6]:
# Creates folder if it doesn't exist
if not os.path.exists('output_3D_1'):
    os.mkdir('output_3D_1')
else:
    files = glob.glob('output_3D_1/*.png')
    for f in files:
        os.remove(f)

# Disable inline printing to prevent all graphs from being shown
pos, vel = initial()

# Start loop
for i in tqdm.tqdm(range(Nt)):
    acc = acceleration(pos, vel, m, h, k, n, lmbda, nu)
    vel += acc * dt
    pos += vel * dt
    rho = density(pos, pos, m, h)

    # Plot (using 3D plotting)
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, projection='3d')

    plt.cla()
    cval = np.minimum((rho - 3) / 3, 1).flatten()

    # 3D scatter plot
    ax.scatter(pos[:, 0], pos[:, 1], pos[:, 2], c=cval, cmap=plt.cm.autumn, s=5, alpha=0.75) #color is changing based on density

    # Set plot limits
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_zlim(-2.5, 2.5)

    # Save plot
    plt.savefig(f'output_3D_1/{i}.png')
    plt.close()

  0%|          | 0/800 [00:00<?, ?it/s]

100%|██████████| 800/800 [01:30<00:00,  8.84it/s]


#### Animation

In [7]:
import re
# Create video from the saved frames
img_array = []
imgs_list = glob.glob('output_3D_1/*.png')

# Sort images based on the numerical part of the filename
lsorted = sorted(imgs_list, key=lambda x: int(re.search(r'(\d+)', os.path.basename(x)).group()))

# Read and store the frames
for filename in tqdm.tqdm(lsorted):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

# Create VideoWriter object to save the video
out = cv2.VideoWriter('simulation_3D_1.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

# Write frames to the video
for i in tqdm.tqdm(range(len(img_array))):
    out.write(img_array[i])

out.release()

print("Video saved as simulation_3D_1.mp4")

  0%|          | 0/800 [00:00<?, ?it/s]

100%|██████████| 800/800 [00:05<00:00, 149.22it/s]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 800/800 [00:01<00:00, 737.82it/s]

Video saved as simulation_3D_1.mp4
